In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        #print(dirname)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import cv2
import matplotlib.pyplot as plt
#import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
import zipfile

with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/test.zip","r") as z:
    z.extractall(".")

In [ ]:
train_X=[]
train_y=[]
main_dir = "/kaggle/working/"
train_dir = "train"
path=os.path.join(main_dir,train_dir)
for i in os.listdir(path):
    category=i.split(".")[0]
    img = cv2.imread(os.path.join(path,i),0)
    new_img=cv2.resize(img,dsize=(100,100))
    train_X.append(new_img)
    train_y.append(category)

**Now I will encode the "train_y" list to 0 if cat and 1 if dog**

In [ ]:
train_y=pd.DataFrame(train_y)
di={0:{'cat': 0, "dog": 1}}
train_y=train_y.replace(di)

In [ ]:
train_y.head()

In [ ]:
train_y=np.array(train_y)
train_X=np.array(train_X).reshape(-1,100,100,1)

In [ ]:
train_X=train_X/255.0
train_X.shape

Now I will Appply the CNN model

In [ ]:
import tensorflow as tf
import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,Conv2D, MaxPooling2D,BatchNormalization

In [ ]:
def my_model():
        model=Sequential()
        model.add(keras.Input(shape=(train_X.shape[1:])))
        model.add(Conv2D(128,3,padding='same',activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(2))
        model.add(Conv2D(128,3,padding='valid',activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(2))
        model.add(Conv2D(64,3,padding='same',activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(2))
        model.add(Conv2D(64,3,padding='same',activation='relu',input_dim=(train_X.shape[1:])))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(2))
        model.add(Conv2D(32,3,padding='same',activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(2))
        model.add(Conv2D(16,3,padding='same',activation='relu'))
        model.add(MaxPooling2D(2))
        model.add(Conv2D(16,3,padding='same',activation='relu'))
        model.add(BatchNormalization())
        model.add(Flatten())
        model.add(Dropout(0.25))
        model.add(Dense(100,activation='relu'))
        model.add(Dense(1,activation='sigmoid'))
        return model

In [ ]:
model=my_model()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
model.fit(train_X,train_y,epochs=15,batch_size=32,validation_split=0.2,verbose=2)

In [ ]:
# This code cell is to inform wether that the images in the test folder are ordered from 1-12500 or not
main_dir = "/kaggle/working/"
test_dir = "test"
path=os.path.join(main_dir,test_dir)
for i in os.listdir(path):
    category=i.split(".")[0]
    print(category)
    break
#Which is not so I need to reorder the images from 1-12500

In [ ]:
test_X=[]
test_y=[]
main_dir = "/kaggle/working/"
test_dir = "test"
path=os.path.join(main_dir,test_dir)
for i in os.listdir(path):
    category=i.split(".")[0]
    img = cv2.imread(os.path.join(path,i),0)
    new_img=cv2.resize(img,dsize=(100,100))
    test_X.append(new_img)
    test_y.append(category)

In [ ]:
test_y=list(map(int,test_y))
test_X=np.array(test_X).reshape(-1,100,100,1)
test_y=np.array(test_y)

In [ ]:
sorted_order= test_y.argsort()
test_y = test_y[sorted_order[::]]
test_X = test_X[sorted_order[::]]

In [ ]:
predictions=model.predict(test_X)

In [ ]:
predictions

In [ ]:
my_submission = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')

In [ ]:
my_submission['label'] = predictions


In [ ]:
my_submission['label'] = my_submission['label'].map(lambda x: 1 if x >= 0.5 else 0 )

In [ ]:
my_submission.tail()


In [ ]:
my_submission.to_csv('my_submission.csv', index=False)


In [ ]:
my_submission